In [ ]:
!pip install transformers datasets scikit-learn torch --quiet


In [ ]:
import pandas as pd
from sklearn.utils import resample

#load dataset
df = pd.read_excel('hate_sample_for_finetune.xlsx')
df = df.dropna(subset=['hate_type_teacher'])

In [ ]:
# Balance classes
df_majority = df[df['hate_type_teacher'] == 'sarcasm-based hate']
df_minority = df[df['hate_type_teacher'] != 'sarcasm-based hate']

# Oversample minority classes to match majority count
df_minority_upsampled = resample(
    df_minority,
    replace=True,
    n_samples=len(df_majority),
    random_state=42
)

In [ ]:
# Combine and shuffle
df_balanced = pd.concat([df_majority, df_minority_upsampled])
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print(df_balanced['hate_type_teacher'].value_counts())

hate_type_teacher
sarcasm-based hate        1267
metaphor-based hate        530
meme-language hate         321
humor-based hate           221
political framing hate     195
Name: count, dtype: int64


In [ ]:
 # Create mapping of hate_type_teacher text labels to integer ids
label_mapping = {label: idx for idx, label in enumerate(df_balanced['hate_type_teacher'].unique())}
df_balanced['hate_type_teacher_int'] = df_balanced['hate_type_teacher'].map(label_mapping)
print("Label mapping:", label_mapping)

Label mapping: {'metaphor-based hate': 0, 'sarcasm-based hate': 1, 'humor-based hate': 2, 'meme-language hate': 3, 'political framing hate': 4}


In [ ]:
# Split into train and validation sets (90% train, 10% val)
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_balanced['clean_text'], df_balanced['hate_type_teacher_int'], test_size=0.2, random_state=42,stratify=df_balanced['hate_type_teacher_int'])


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


model.safetensors:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

In [ ]:
import torch

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True)  # reset index for safe indexing
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels.iloc[idx]), dtype=torch.long)
        return item
    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)


In [ ]:
trainer.train()

Step,Training Loss
500,1.334300


TrainOutput(global_step=635, training_loss=1.3242675060362328, metrics={'train_runtime': 14.5025, 'train_samples_per_second': 698.846, 'train_steps_per_second': 43.786, 'total_flos': 1057254457140.0, 'train_loss': 1.3242675060362328, 'epoch': 5.0})

In [ ]:
model.save_pretrained('./tinybert-hate-subtypes')
tokenizer.save_pretrained('./tinybert-hate-subtypes')


('./tinybert-hate-subtypes/tokenizer_config.json',
 './tinybert-hate-subtypes/special_tokens_map.json',
 './tinybert-hate-subtypes/vocab.txt',
 './tinybert-hate-subtypes/added_tokens.json',
 './tinybert-hate-subtypes/tokenizer.json')

In [ ]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 1.28767728805542, 'eval_runtime': 0.1503, 'eval_samples_per_second': 3372.333, 'eval_steps_per_second': 212.849, 'epoch': 5.0}



*  eval_loss: The error on validation data; lower is better. Around 1.1 shows the model is learning but can improve.
*   eval_runtime: Time taken to evaluate (~0.16 seconds here).

*   eval_samples_per_second: Speed of evaluation (1273 samples/second).
*   eval_steps_per_second: Validation batches processed per second (83 steps/second).

*   epoch: Which training cycle this evaluation is for (3rd epoch here).








In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Inference par predicted labels aur true labels nikal lo (example below)

predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(axis=1)  # model ke predicted labels
true_labels = predictions.label_ids  # ground truth labels

print("Accuracy:", accuracy_score(true_labels, pred_labels))
print("Classification Report:\n", classification_report(true_labels, pred_labels))


Accuracy: 0.5009861932938856
Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       106
           1       0.50      1.00      0.67       254
           2       0.00      0.00      0.00        44
           3       0.00      0.00      0.00        64
           4       0.00      0.00      0.00        39

    accuracy                           0.50       507
   macro avg       0.10      0.20      0.13       507
weighted avg       0.25      0.50      0.33       507



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
